### import requirements

In [10]:
import requests, pandas as pd
from bs4 import BeautifulSoup

### scrape contents of article "List of postal codes of Canada"

In [27]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page, 'html.parser')

### parse table data for postal code data

In [30]:
table = soup.find('table', attrs={'class': 'wikitable sortable'})
postal_code_data = table.find_all('tr')

### place postal code, borough, and neighborhood into dataframe

In [59]:
postal_codes = []

for row in results:
    contents = row.findAll('td')
    if len(contents) > 0: 
        postal_codes.append([char.text.strip('\n') for char in contents])
    

df = pd.DataFrame(postal_codes, columns=['PostalCode', 'Borough', 'Neighbourhood']).reset_index(drop=True)
df = df.loc[df.Borough.ne('Not assigned')].reset_index(drop=True)
df_final = df.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna()))).reset_index(drop=True)

df_final.shape

(103, 3)

### import coordinates file

In [68]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

### map coordinates to postal codes

In [71]:
df_final['Latitude'] = df_final['PostalCode'].map(coordinates.set_index('Postal Code')['Latitude']).fillna(0)
df_final['Longitude'] = df_final['PostalCode'].map(coordinates.set_index('Postal Code')['Longitude']).fillna(0)
df_final.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Guildwood, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848
